# Collaborative filtering for 'implicit feedback' data
** *
This projects consists in building a recommender system to recommend songs according to the history of tracks played by a given user from subset of the [MillionSong TasteProfile data](https://labrosa.ee.columbia.edu/millionsong/tasteprofile) that was made into a [Kaggle competition](https://www.kaggle.com/c/msdchallenge), using [Hierarchical Poisson Factorization](https://arxiv.org/abs/1311.1704). Unlike recommendations based on movie ratings, for example, recommending based on listening activity is harder as it is only an indirect measure of user preference, and doesn’t usually signal user dislikes.

HPF (Hierarchical Poisson Factorization) is a probabilistic model that tries to factorize the user-item interaction count matrix as the product of two lower dimensional matrices, just like regular factorization methods used with explicit feedback data, but taking this product as the parameter of a Poisson random variable (thus the model likelihood optimization is different than minimizing the sum of least squares). Additionally, the user-attribute and item-attribute matrices are given a Bayesian Gamma prior, making them non-negative and adjusting for the user activity level and song popularity.

Unlike other methods such as BPR (Bayesian Personalized Ranking) or weighted-implicit ALS, it only requires iterating over the data for which an interaction was observed and not over data for which no interaction was observed (i.e. it doesn’t iterate over songs not played by the user), thus being more scalable, and at the same time producing better results.

The implementation here is based on the paper _Gopalan, P., Hofman, J. M., & Blei, D. M. (2013). Scalable recommendation with poisson factorization. arXiv preprint arXiv:1311.1704._, and is implemented using PyMC3 through Maximum-a-Posteriori, so speed is not as great as the authors' original coordinate ascent optimization algorithm.

It’s possible to extend this model to make use of song side information, such as artist, tags, genre, and others, but that increases the model complexity and computational time a lot without bringing too much of an improvement (as seen in the paper _Gopalan, P. K., Charlin, L., & Blei, D. (2014). Content-based recommendations with poisson factorization. In Advances in Neural Information Processing Systems (pp. 3176-3184)._)
** *
# Sections

[1. Loading the data](#p1)

[2. Implementing the model](#p2)

[3. Checking some recommendations](#p3)
** *
<a id="p1"></a>
## 1. Loading the data

Reading and reindexing the data:

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, pymc3 as pm, theano

playcounts=list()
user_id_to_int=dict()
user_int_to_id=dict()
song_id_to_int=dict()
song_int_to_id=dict()
cnt_users=0
cnt_songs=0

def parse_line(line):
    global playcounts, cnt_users, cnt_songs
    user,song,playcount = line.decode('utf-8').split('\t')
    if user not in user_id_to_int:
        user_id_to_int[user]=cnt_users
        user_int_to_id[cnt_users]=user
        cnt_users+=1
    if song not in song_int_to_id:
        song_int_to_id[song]=cnt_songs
        song_int_to_id[cnt_songs]=song
        cnt_songs+=1
    
    user=user_id_to_int[user]
    song=song_int_to_id[song]
    playcount=int(playcount.strip())
    
    playcounts.append((user, song, playcount))

with open('D:\\Downloads\\millionsong\\kaggle_visible_evaluation_triplets.txt','rb') as f:
    for line in f:
        parse_line(line)
              
print(cnt_users)
print(cnt_songs)
print(len(playcounts))

110000
163206
1450933


_Note that an algorithm like implicit-ALS would require constructing a matrix with more than 10^10 entries with this dataset, thus not feasible on a cheap laptop._

In [2]:
del user_id_to_int
del user_int_to_id
del song_id_to_int

The dataset doesn't contain timestamps so I'll make a random train-test split:

In [3]:
from sklearn.model_selection import train_test_split

playcounts=pd.DataFrame(playcounts, columns=['UserId', 'SongId', 'Playcount'])

train, test = train_test_split(playcounts, test_size=.2, random_state=1)

users_train=set(train.UserId)
items_train=set(train.SongId)
test=test.loc[test.UserId.isin(users_train)]
test=test.loc[test.UserId.isin(items_train)]
test=test.loc[test.Playcount>1]
del users_train
del items_train

print(train.shape)
print(test.shape)
train.head()

(1160746, 3)
(115511, 3)


,UserId,SongId,Playcount
745579,56512,49759,1
688265,52114,434,3
1425112,108044,112662,1
231922,17542,325,1
1159262,87716,2623,6


<a id="p2"></a>
## 2. Implementing the model

PyMC3 Implementation. The hyperparameters are the ones suggested in the paper:

In [4]:
# hyperparameters
a=.3
a_=.3
c=.3
c_=.3

b_=1.0
d_=1.0

# number of factors
k = 25

In [5]:
with pm.Model() as pmf2:
    user_activity=pm.Gamma('user_activity',a_,a_/b_, shape = (cnt_users,1) )
    user_prior=theano.tensor.tile(user_activity, k, ndim=2)
    theta=pm.Gamma('theta',a,user_prior, shape=(cnt_users,k))
    
    item_pop=pm.Gamma('item_pop', c_,c_/d_, shape = (cnt_songs,1) )
    item_prior=theano.tensor.tile(item_pop, k, ndim=2)
    beta=pm.Gamma('beta', c, item_prior, shape=(cnt_songs,k))
    
    xhat=theano.tensor.sum(theta[playcounts.UserId] * beta[playcounts.SongId], axis=1)
    R=pm.Poisson('R',mu=xhat, observed=playcounts.Playcount.astype('float32'))
    HPF=pm.find_MAP()

logp = 7.2406e+06, ||grad|| = 12,655: 100%|████| 80/80 [14:36<00:00,  7.18s/it]   


Now extracting the results:

In [6]:
theta_star=HPF['theta']
beta_star=HPF['beta'].T

<a id="p3"></a>
## 3. Checking some recommendations

Now examining the Top-20 recommended songs for some users. The list of song names, artist, and other side information is also made available:

In [7]:
song_info=pd.read_table('D:\\Downloads\\millionsong\\unique_tracks.txt', sep='<SEP>', engine='python',
              header=None, names=['TrackId', 'SongId', 'Artist', 'Title'])
del song_info['TrackId']
song_info.head()

,SongId,Artist,Title
0,SOQMMHC12AB0180CB8,Faster Pussy cat,Silent Night
1,SOVFVAK12A8C1350D9,Karkkiautomaatti,Tanssi vaan
2,SOGTUKN12AB017F4F1,Hudson Mohawke,No One Could Ever
3,SOBNYVR12A8C13558C,Yerba Brava,Si Vos QuerÃ©s
4,SOHSBXH12A8C13B0DF,Der Mystic,Tangle Of Aspens


In [8]:
def top_n(user_id, n=20):
    global theta_star, beta_star, song_info
    recommended_list = np.argsort(theta_star[user_id].dot(beta_star))
    recommended_list = [song_int_to_id[song] for song in recommended_list[:n]]
    recommended_list = pd.DataFrame(pd.Series(recommended_list), columns=['SongId'])
    return pd.merge(recommended_list, song_info, on='SongId', how='left')


top_n(0)

,SongId,Artist,Title
0,SOBONKR12A58A7A7E0,Dwight Yoakam,You're The One
1,SOEKQWZ12A6D4FB11F,Danzig,Dominion
2,SOYRGSH12AB017F24D,Ricky Martin,It's Alright
3,SOHCBUL12AB0181266,Rose Elinor Dougall,Start/Stop/Synchro
4,SOZUEFV12A8C141169,Pacha Massive,Drive
5,SOHEPPO12A8C14120A,YACHT,Im In Love With A Ripper (Party Mix)
6,SOZMHYF12AB0187114,Morningwood,Hot Tonight
7,SOMCJBU12AB017BA6E,Screamin' Jay Hawkins,Guess Who?
8,SOVOQCN12AF72A134A,Screamin' Jay Hawkins,Voodoo Priestess
9,SOWMBXP12AB017BF25,Screamin' Jay Hawkins,I Don't Know


In [9]:
top_n(123)

,SongId,Artist,Title
0,SOBONKR12A58A7A7E0,Dwight Yoakam,You're The One
1,SOTFHFY1288D3EB5CB,Flo Rida,Elevator [Feat. Timbaland] (Album Version)
2,SOAKMDU12A8C1346A9,The Postal Service,Such Great Heights
3,SODLLYS12A8C13A96B,The Script,Breakeven
4,SOMGIYR12AB0187973,Panic At The Disco,Behind The Sea [Live In Chicago]
5,SOQKBIT12A6D4FA8BF,Akon / Snoop Dogg,I Wanna Love You
6,SOZEBAZ12AF72A80C8,Thursday,Voices On A String (Album Version)
7,SONETQG12AF72A16FF,Polly Paulusma,Give It Back
8,SOHFGKG12A6701C429,Sheryl Crow,Picture
9,SOZHUUI12A6701D7B6,The All-American Rejects,Swing_ Swing


In [10]:
top_n(53000)

,SongId,Artist,Title
0,SOBONKR12A58A7A7E0,Dwight Yoakam,You're The One
1,SOTFHFY1288D3EB5CB,Flo Rida,Elevator [Feat. Timbaland] (Album Version)
2,SOAKMDU12A8C1346A9,The Postal Service,Such Great Heights
3,SODLLYS12A8C13A96B,The Script,Breakeven
4,SOMGIYR12AB0187973,Panic At The Disco,Behind The Sea [Live In Chicago]
5,SOQKBIT12A6D4FA8BF,Akon / Snoop Dogg,I Wanna Love You
6,SOZEBAZ12AF72A80C8,Thursday,Voices On A String (Album Version)
7,SONETQG12AF72A16FF,Polly Paulusma,Give It Back
8,SOHFGKG12A6701C429,Sheryl Crow,Picture
9,SOZHUUI12A6701D7B6,The All-American Rejects,Swing_ Swing


In [11]:
top_n(100000)

,SongId,Artist,Title
0,SOBONKR12A58A7A7E0,Dwight Yoakam,You're The One
1,SOEKQWZ12A6D4FB11F,Danzig,Dominion
2,SOYRGSH12AB017F24D,Ricky Martin,It's Alright
3,SOHCBUL12AB0181266,Rose Elinor Dougall,Start/Stop/Synchro
4,SOZUEFV12A8C141169,Pacha Massive,Drive
5,SOHEPPO12A8C14120A,YACHT,Im In Love With A Ripper (Party Mix)
6,SOZMHYF12AB0187114,Morningwood,Hot Tonight
7,SOMCJBU12AB017BA6E,Screamin' Jay Hawkins,Guess Who?
8,SOVOQCN12AF72A134A,Screamin' Jay Hawkins,Voodoo Priestess
9,SOWMBXP12AB017BF25,Screamin' Jay Hawkins,I Don't Know


Unfortunately, for implicit data such as playcounts, there is no intuitive metric such as the root mean squared error or average rating of Top-N recommendations to compare with. Nevertheless, it's possible to do some common sense checks such as checking the mean prediction for songs that were in the test set and compare it to randomly selected songs:

In [36]:
test['Predicted']=test.apply(lambda x: theta_star[int(x['UserId'])].dot(beta_star[:,int(x['SongId'])]), axis=1)
test['Predicted'].loc[test.Predicted.map(lambda x: np.isinf(x))]=1000.0
test['Predicted']=test.Predicted.fillna(1000.0)
print(test.Predicted.mean())
print(np.corrcoef([test.Predicted, test.Playcount])[0,1])
test.head()

C:\Users\david\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


173.61525540122125
0.0168017628183


,UserId,SongId,Playcount,Predicted,RandomSongId,PredictedRandom
689569,52215,3667,5,2296.991455,29635,1.073157e-01
2399,184,2121,3,0.000004,86271,2.090896e-06
948969,71943,2666,11,0.000000,113524,0.000000e+00
1214563,91846,2605,2,0.000915,67243,6.211372e-07
1448562,109821,51636,2,0.000000,2,0.000000e+00


In [39]:
test['RandomSongId']=np.random.randint(cnt_songs, size=test.shape[0])
test['PredictedRandom']=test.apply(lambda x: theta_star[int(x['UserId'])].dot(beta_star[:,int(x['RandomSongId'])]), axis=1)
test['PredictedRandom'].loc[test.PredictedRandom.map(lambda x: np.isinf(x))]=1000.0
test['PredictedRandom']=test.PredictedRandom.fillna(1000.0)
print(test.PredictedRandom.mean())

C:\Users\david\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


70.06465376460116


It's also possible to see how many of the hold-out songs for a given user would be in his or her Top-N recommended list (Precision@K), but this is not a very good metric as many users will only have 1 or 2 songs in the hold-out set and this metric can only reach very low numbers:

In [13]:
np.random.seed(1)

users_test_sample=set(test.UserId)
users_test_sample=[u for u in users_test_sample]
users_test_sample=np.random.choice(users_test_sample, size=200, replace=False)
users_test_sample=list(users_test_sample)
already_played=train.loc[train.UserId.isin(users_test_sample)]
already_played=already_played.groupby('UserId')['SongId'].agg(lambda x: set(x))

def top_k_user(user_id, k=40):
    global theta_star, beta_star, already_played
    reclist=np.argsort(theta_star[user_id].dot(beta_star))
    top_k=list()
    for item in reclist:
        if item not in already_played.loc[user_id]:
            top_k.append(item)
        if len(top_k)==k:
            break
    return top_k

def precision(rec_list, played_list):
    rec_list=set(rec_list)
    overlap=rec_list.intersection(played_list)
    return len(overlap)/len(rec_list)

users_test_sample_set=set(users_test_sample)
songs_holdout=test.loc[test.UserId.isin(users_test_sample_set)]
songs_holdout=songs_holdout.groupby('UserId')['SongId'].agg(lambda x: set(x)).to_frame()
songs_holdout.columns=['Played']
test_eval=pd.DataFrame(pd.Series(list(users_test_sample)), columns=['UserId'])
test_eval=pd.merge(test_eval, songs_holdout, left_on='UserId', right_index=True)
test_eval['RecList']=test_eval.UserId.map(top_k_user)

test_eval['PrecisionAt20']=test_eval.apply(lambda x: precision(x['RecList'][:20],x['Played']), axis=1)
test_eval['PrecisionAt25']=test_eval.apply(lambda x: precision(x['RecList'][:25],x['Played']), axis=1)
test_eval['PrecisionAt30']=test_eval.apply(lambda x: precision(x['RecList'][:30],x['Played']), axis=1)
test_eval['PrecisionAt35']=test_eval.apply(lambda x: precision(x['RecList'][:35],x['Played']), axis=1)
test_eval['PrecisionAt40']=test_eval.apply(lambda x: precision(x['RecList'][:40],x['Played']), axis=1)
test_eval.describe()

,UserId,PrecisionAt20,PrecisionAt25,PrecisionAt30,PrecisionAt35,PrecisionAt40
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,54064.360000,0.000250,0.000200,0.000167,0.000143,0.000125
std,30880.855653,0.003536,0.002828,0.002357,0.002020,0.001768
min,113.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,26925.750000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,53140.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,79244.250000,0.000000,0.000000,0.000000,0.000000,0.000000
max,109630.000000,0.050000,0.040000,0.033333,0.028571,0.025000
